<h1 style="font-size:40px;"><center>Exercise V:<br> GANs
</center></h1>

## Short summary
In this exercise, we will design a generative network to generate the last rgb image given the first image. These folder has **three files**: 
- **configGAN.py:** this involves definitions of all parameters and data paths
- **utilsGAN.py:** includes utility functions required to grab and visualize data 
- **runGAN.ipynb:** contains the script to design, train and test the network 

Make sure that before running this script, you created an environment and **installed all required libraries** such 
as keras.

## The data
There exists also a subfolder called **data** which contains the traning, validation, and testing data each has both RGB input images together with the corresponding ground truth images.


## The exercises
As for the previous lab all exercises are found below.


## The different 'Cells'
This notebook contains several cells with python code, together with the markdown cells (like this one) with only text. Each of the cells with python code has a "header" markdown cell with information about the code. The table below provides a short overview of the code cells. 

| #  |  CellName | CellType | Comment |
| :--- | :-------- | :-------- | :------- |
| 1 | Init | Needed | Sets up the environment|
| 2 | Ex | Exercise 1| A class definition of a network model  |
| 3 | Loading | Needed | Loading parameters and initializing the model |
| 4 | Stats | Needed | Show data distribution | 
| 5 | Data | Needed | Generating the data batches |
| 6 | Debug | Needed | Debugging the data |
| 7 | Device | Needed | Selecting CPU/GPU |
| 8 | Init | Needed | Sets up the timer and other neccessary components |
| 9 | Training | Exercise 1-2 | Training the model   |
| 10 | Testing | Exercise 1-2| Testing the  method   |  


In order for you to start with the exercise you need to run all cells. It is important that you do this in the correct order, starting from the top and continuing with the next cells. Later when you have started to work with the notebook it may be easier to use the command "Run All" found in the "Cell" dropdown menu.

## Writing the report

There is no need to provide any report. However, implemented network architecuture and observed experimental results must be presented as a short presentation in the last lecture, May 28.

1) We first start with importing all required modules

In [1]:
import os
from configGAN import *
cfg = flying_objects_config()
if cfg.GPU >=0:
    print("creating network model using gpu " + str(cfg.GPU))
    os.environ['CUDA_VISIBLE_DEVICES'] = str(cfg.GPU)
elif cfg.GPU >=-1:
    print("creating network model using cpu ")  
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
    os.environ["CUDA_VISIBLE_DEVICES"] = ""

import tensorflow as tf
from tensorflow import keras
from utilsGAN import *
from sklearn.metrics import confusion_matrix
# import seaborn as sns
from datetime import datetime
import imageio
from skimage import img_as_ubyte

import pprint
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv3D, Conv2D, Conv1D, Convolution2D, Deconvolution2D, Cropping2D, UpSampling2D
from keras.layers import Input, Conv2DTranspose, ConvLSTM2D, TimeDistributed
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers import Concatenate, concatenate, Reshape
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.layers import Input, merge
from keras.regularizers import l2
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout, LeakyReLU
import keras.backend as kb


creating network model using gpu 0


2) Here, we have the network model class definition. In this class, the most important functions are **build_generator()** and **build_discriminator()**. As defined in the exercises section, your task is to update the both network architectures defined in these functions.

In [2]:
class GANModel():
    def __init__(self, batch_size=32, inputShape=(64, 64, 3), dropout_prob=0.25): 
        self.batch_size = batch_size
        self.inputShape = inputShape
        self.dropout_prob = dropout_prob

        # Calculate the shape of patches
        patch = int(self.inputShape[0] / 2**4)
        self.disc_patch = (patch, patch, 1)
  
        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mse', optimizer=Adam(0.0002, 0.5),metrics=['accuracy'])
 
        # Build the generator
        self.generator = self.build_generator()

        # Input images and their conditioning images
        first_frame = Input(shape=self.inputShape)
        last_frame = Input(shape=self.inputShape)

        # By conditioning on the first frame generate a fake version of the last frame
        fake_last_frame = self.generator(first_frame)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False
        
        # Discriminators determines validity of fake and condition first image pairs
        valid = self.discriminator([fake_last_frame, first_frame])

        self.combined = Model(inputs=[last_frame, first_frame], outputs=[valid, fake_last_frame])
        self.combined.compile(loss=['mse', 'mae'], # mean squared and mean absolute errors
                              loss_weights=[1, 100],
                              optimizer=Adam(0.0002, 0.5))

    def build_generator(self):
        inputs = Input(shape=self.inputShape)
        
        conv1= Conv2D(filters=32,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(inputs)
        conv1 = BatchNormalization(momentum=0.8)(conv1)
        conv1= Conv2D(filters=32,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv1)
        conv1 = BatchNormalization(momentum=0.8)(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        
        conv2= Conv2D(filters=64,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(pool1)
        conv2 = BatchNormalization(momentum=0.8)(conv2)
        conv2= Conv2D(filters=64,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv2)
        conv2 = BatchNormalization(momentum=0.8)(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        
        conv3= Conv2D(filters=128,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(pool2)
        conv3 = BatchNormalization(momentum=0.8)(conv3)
        conv3= Conv2D(filters=128,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv3)
        conv3 = BatchNormalization(momentum=0.8)(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
        
        conv4= Conv2D(filters=256,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(pool3)
        conv4 = BatchNormalization(momentum=0.8)(conv4)
        conv4= Conv2D(filters=256,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv4)
        conv4 = BatchNormalization(momentum=0.8)(conv4)
        drop4 = Dropout(0.5)(conv4)
        '''pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
        
        conv5= Conv2D(filters=256,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(pool4)
        conv5= Conv2D(filters=256,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv5)
        #pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        drop5 = Dropout(0.5)(conv5)
        
        
        up6 = UpSampling2D(size=(2, 2))(drop5)
        up6 = Conv2D(filters=256,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(up6)
        up6 = Concatenate(axis=3)([drop4, up6])
        conv6 = Conv2D(filters=256,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(up6)
        conv6 = Conv2D(filters=256,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv6)'''

        up7 = UpSampling2D(size=(2, 2))(conv4)
        up7 = Conv2D(filters=256,kernel_size=2,activation='relu',padding='same',kernel_initializer='he_normal')(up7)
        up7 = BatchNormalization(momentum=0.8)(up7)
        up7 = Concatenate(axis=3)([conv3, up7])
        conv7 = Conv2D(filters=256,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(up7)
        conv7 = BatchNormalization(momentum=0.8)(conv7)
        conv7 = Conv2D(filters=256,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv7)
        conv7 = BatchNormalization(momentum=0.8)(conv7)

        up8 = UpSampling2D(size=(2, 2))(conv7)
        up8 = Conv2D(filters=128,kernel_size=2,activation='relu',padding='same',kernel_initializer='he_normal')(up8)
        up8 = BatchNormalization(momentum=0.8)(up8)
        up8 = Concatenate(axis=3)([conv2, up8])
        conv8 = Conv2D(filters=128,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(up8)
        conv8 = BatchNormalization(momentum=0.8)(conv8)
        conv8 = Conv2D(filters=128,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv8)

        up9 = UpSampling2D(size=(2, 2))(conv8)
        up9 = Conv2D(filters=64,kernel_size=2,activation='relu',padding='same',kernel_initializer='he_normal')(up9)
        up9 = BatchNormalization(momentum=0.8)(up9)
        up9 = Concatenate(axis=3)([conv1, up9])
        conv9 = Conv2D(filters=64,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(up9)
        conv9 = BatchNormalization(momentum=0.8)(conv9)
        conv9 = Conv2D(filters=64,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv9)
        conv9 = BatchNormalization(momentum=0.8)(conv9)
        conv9 = Conv2D(filters=32,kernel_size=3,activation='relu',padding='same',kernel_initializer='he_normal')(conv9)
        conv9 = BatchNormalization(momentum=0.8)(conv9)
        
        nbr_img_channels = self.inputShape[2]

        outputs = Conv2D(nbr_img_channels, (1, 1), activation='sigmoid')(conv9)

        model = Model(inputs=inputs, outputs=outputs, name='Generator')
        model.summary()
 
        '''inputs = Input(shape=self.inputShape)
        print(inputs.shape)
 
        down1 = Conv2D(32, (3, 3),padding='same')(inputs)
        down1 = Activation('relu')(down1) 
        down1_pool = MaxPooling2D((2, 2), strides=(2, 2))(down1)
         
        down2 = Conv2D(64, (3, 3), padding='same')(down1_pool)
        down2 = Activation('relu')(down2) 
         

        up1 = UpSampling2D((2, 2))(down2)
        up1 = concatenate([down1, up1], axis=3)
        up1 = Conv2D(256, (3, 3), padding='same')(up1) 
        up1 = Activation('relu')(up1) 
        
        
        up2 = Conv2D(256, (3, 3), padding='same')(up1) 
        up2 = Activation('relu')(up2) 
        
        nbr_img_channels = self.inputShape[2]
        outputs = Conv2D(nbr_img_channels, (1, 1), activation='sigmoid')(up2)

        model = Model(inputs=inputs, outputs=outputs, name='Generator')
        model.summary()'''
        
        return model

    def build_discriminator(self):
  
        last_img = Input(shape=self.inputShape)
        first_img = Input(shape=self.inputShape)

        # Concatenate image and conditioning image by channels to produce input
        combined_imgs = Concatenate(axis=-1)([last_img, first_img])
  
        d1 = Conv2D(32, (3, 3), strides=2, padding='same')(combined_imgs) 
        d1 = Activation('relu')(d1) 
        d2 = Conv2D(64, (3, 3), strides=2, padding='same')(d1)
        d2 = Activation('relu')(d2) 
        d3 = Conv2D(128, (3, 3), strides=2, padding='same')(d2)
        d3 = Activation('relu')(d3) 
         
        validity = Conv2D(1, (3, 3), strides=2, padding='same')(d3)

        model = Model([last_img, first_img], validity)
        model.summary()

        return model

3) We import the network **hyperparameters** and build a simple network by calling the class introduced in the previous step. Please note that to change the hyperparameters, you just need to change the values in the file called **configPredictor.py.**

In [3]:
image_shape = (cfg.IMAGE_HEIGHT, cfg.IMAGE_WIDTH, cfg.IMAGE_CHANNEL)
modelObj = GANModel(batch_size=cfg.BATCH_SIZE, inputShape=image_shape,
                                 dropout_prob=cfg.DROPOUT_PROB)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 32, 32, 6)    0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 16, 16, 32)   1760        concatenate[0][0]     

4) We call the utility function **show_statistics** to display the data distribution. This is just for debugging purpose.

In [4]:
#### show how the data looks like
show_statistics(cfg.training_data_dir, fineGrained=False, title=" Training Data Statistics ")
show_statistics(cfg.validation_data_dir, fineGrained=False, title=" Validation Data Statistics ")
show_statistics(cfg.testing_data_dir, fineGrained=False, title=" Testing Data Statistics ")


######################################################################
##################### Training Data Statistics #####################
######################################################################
total image number 	 10817
total class number 	 3
class square 	 3488 images
class triangle 	 3703 images
class circular 	 3626 images
######################################################################

######################################################################
##################### Validation Data Statistics #####################
######################################################################
total image number 	 2241
total class number 	 3
class circular 	 713 images
class square 	 783 images
class triangle 	 745 images
######################################################################

######################################################################
##################### Testing Data Statistics #####################
##########################

5) We now create batch generators to get small batches from the entire dataset. There is no need to change these functions as they already return **normalized inputs as batches**.

In [5]:
nbr_train_data = get_dataset_size(cfg.training_data_dir)
nbr_valid_data = get_dataset_size(cfg.validation_data_dir)
nbr_test_data = get_dataset_size(cfg.testing_data_dir)
train_batch_generator = generate_lastframepredictor_batches(cfg.training_data_dir, image_shape, cfg.BATCH_SIZE)
valid_batch_generator = generate_lastframepredictor_batches(cfg.validation_data_dir, image_shape, cfg.BATCH_SIZE)
test_batch_generator = generate_lastframepredictor_batches(cfg.testing_data_dir, image_shape, cfg.BATCH_SIZE)
print("Data batch generators are created!")

Data batch generators are created!


6) We can visualize how the data looks like for debugging purpose

In [7]:
if cfg.DEBUG_MODE:
    t_x, t_y,_ = next(train_batch_generator)
    print('train_x', t_x.shape, t_x.dtype, t_x.min(), t_x.max())
    print('train_y', t_y.shape, t_y.dtype, t_y.min(), t_y.max()) 
    #plot_sample_lastframepredictor_data_with_groundtruth(t_x, t_y, t_y)
    pprint.pprint (cfg)

train_x (30, 32, 32, 3) float32 0.0 1.0
train_y (30, 32, 32, 3) float32 0.0 1.0
{'BATCH_SIZE': 30,
 'CLASSES': ['square_red',
             'square_green',
             'square_blue',
             'square_yellow',
             'triangle_red',
             'triangle_green',
             'triangle_blue',
             'triangle_yellow',
             'circular_red',
             'circular_green',
             'circular_blue',
             'circular_yellow'],
 'DATA_AUGMENTATION': True,
 'DEBUG_MODE': True,
 'DROPOUT_PROB': 0.5,
 'GPU': 0,
 'IMAGE_CHANNEL': 3,
 'IMAGE_HEIGHT': 32,
 'IMAGE_WIDTH': 32,
 'LEARNING_RATE': 0.01,
 'LR_DECAY_FACTOR': 0.1,
 'NUM_EPOCHS': 5,
 'PRINT_EVERY': 20,
 'SAVE_EVERY': 1,
 'SEQUENCE_LENGTH': 10,
 'testing_data_dir': '../data/FlyingObjectDataset_10K/testing',
 'training_data_dir': '../data/FlyingObjectDataset_10K/training',
 'validation_data_dir': '../data/FlyingObjectDataset_10K/validation'}


7) Start timer and init matrices

In [8]:
start_time = datetime.now()
# Adversarial loss ground truths
valid = np.ones((cfg.BATCH_SIZE,) + modelObj.disc_patch)
fake = np.zeros((cfg.BATCH_SIZE,) + modelObj.disc_patch)
# log file
output_log_dir = "./logs/{}".format(datetime.now().strftime("%Y%m%d-%H%M%S"))
if not os.path.exists(output_log_dir):
    os.makedirs(output_log_dir)

8) We can now feed the training and validation data to the network. This will train the network for **some epochs**. Note that the epoch number is also predefined in the file called **configGAN.py.**

In [10]:
import imageio
import matplotlib.pyplot as plt
from skimage import img_as_ubyte
import numpy as np 

%matplotlib inline


test_first_imgs, test_last_imgs,_ = next(test_batch_generator)

for epoch in range(cfg.NUM_EPOCHS):
    steps_per_epoch = (nbr_train_data // cfg.BATCH_SIZE) 
    for batch_i in range(steps_per_epoch):
        first_frames, last_frames,_= next(train_batch_generator)
        if first_frames.shape[0] == cfg.BATCH_SIZE: 
             
            # Condition on the first frame and generate the last frame
            fake_last_frames = modelObj.generator.predict(first_frames)
            #plt.imshow(fake_last_frames[1])
            print(fake_last_frames.shape)
            #print(tf.keras.backend.mean(fake_last_frames[0]))
            print(np.mean(fake_last_frames[0]))

            # Train the discriminator with combined loss  
            d_loss_real = modelObj.discriminator.train_on_batch([last_frames, first_frames], valid)
            d_loss_fake = modelObj.discriminator.train_on_batch([fake_last_frames, first_frames], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
 
            # Train the generator
            g_loss = modelObj.combined.train_on_batch([last_frames, first_frames], [valid, last_frames])

            elapsed_time = datetime.now() - start_time 
            print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f] time: %s" % (epoch, cfg.NUM_EPOCHS,
                                                                                               batch_i,
                                                                                               steps_per_epoch,
                                                                                               d_loss[0], 
                                                                                               g_loss[0],
                                                                                               elapsed_time))
            # run some tests to check how the generated images evolve during training
            test_fake_last_imgs = modelObj.generator.predict(test_first_imgs)
            test_img_name = output_log_dir + "/gen_img_epoc_" + str(epoch) + ".png"
            merged_img = np.vstack((first_frames[0],last_frames[0],fake_last_frames[0]))
            imageio.imwrite(test_img_name, img_as_ubyte(merged_img)) #scipy.misc.imsave(test_img_name, merged_img)
  

(30, 32, 32, 3)
0.5381139
[Epoch 0/5] [Batch 0/360] [D loss: 0.549909] [G loss: 51.827637] time: 0:00:16.458334
(30, 32, 32, 3)
0.48409462
[Epoch 0/5] [Batch 1/360] [D loss: 0.470331] [G loss: 49.959103] time: 0:00:16.564313
(30, 32, 32, 3)
0.4876071
[Epoch 0/5] [Batch 2/360] [D loss: 0.409592] [G loss: 48.253975] time: 0:00:16.669413
(30, 32, 32, 3)
0.52264667
[Epoch 0/5] [Batch 3/360] [D loss: 0.357141] [G loss: 47.305038] time: 0:00:16.777142
(30, 32, 32, 3)
0.5700949
[Epoch 0/5] [Batch 4/360] [D loss: 0.306269] [G loss: 45.743374] time: 0:00:16.881245
(30, 32, 32, 3)
0.63782007
[Epoch 0/5] [Batch 5/360] [D loss: 0.269805] [G loss: 44.852943] time: 0:00:16.991665
(30, 32, 32, 3)
0.6291484
[Epoch 0/5] [Batch 6/360] [D loss: 0.240555] [G loss: 43.152534] time: 0:00:17.095322
(30, 32, 32, 3)
0.6695692
[Epoch 0/5] [Batch 7/360] [D loss: 0.220810] [G loss: 41.462101] time: 0:00:17.199552
(30, 32, 32, 3)
0.6267225
[Epoch 0/5] [Batch 8/360] [D loss: 0.205313] [G loss: 40.974670] time: 0:00

[Epoch 0/5] [Batch 74/360] [D loss: 0.015724] [G loss: 25.648184] time: 0:00:24.326813
(30, 32, 32, 3)
0.7491582
[Epoch 0/5] [Batch 75/360] [D loss: 0.008585] [G loss: 25.473957] time: 0:00:24.434185
(30, 32, 32, 3)
0.6888735
[Epoch 0/5] [Batch 76/360] [D loss: 0.013013] [G loss: 26.318098] time: 0:00:24.542146
(30, 32, 32, 3)
0.7386187
[Epoch 0/5] [Batch 77/360] [D loss: 0.007275] [G loss: 25.979515] time: 0:00:24.655078
(30, 32, 32, 3)
0.6743868
[Epoch 0/5] [Batch 78/360] [D loss: 0.013529] [G loss: 25.403847] time: 0:00:24.761054
(30, 32, 32, 3)
0.7180272
[Epoch 0/5] [Batch 79/360] [D loss: 0.005862] [G loss: 25.808718] time: 0:00:24.866029
(30, 32, 32, 3)
0.59154725
[Epoch 0/5] [Batch 80/360] [D loss: 0.013414] [G loss: 25.000858] time: 0:00:24.974711
(30, 32, 32, 3)
0.6319538
[Epoch 0/5] [Batch 81/360] [D loss: 0.007359] [G loss: 24.996452] time: 0:00:25.082316
(30, 32, 32, 3)
0.7460494
[Epoch 0/5] [Batch 82/360] [D loss: 0.014791] [G loss: 24.566753] time: 0:00:25.201139
(30, 32,

[Epoch 0/5] [Batch 146/360] [D loss: 0.010696] [G loss: 21.697216] time: 0:00:31.993315
(30, 32, 32, 3)
0.782044
[Epoch 0/5] [Batch 147/360] [D loss: 0.010820] [G loss: 21.727226] time: 0:00:32.098073
(30, 32, 32, 3)
0.7109005
[Epoch 0/5] [Batch 148/360] [D loss: 0.016685] [G loss: 21.685747] time: 0:00:32.202757
(30, 32, 32, 3)
0.7283683
[Epoch 0/5] [Batch 149/360] [D loss: 0.007694] [G loss: 21.817930] time: 0:00:32.313877
(30, 32, 32, 3)
0.66107106
[Epoch 0/5] [Batch 150/360] [D loss: 0.013005] [G loss: 21.938232] time: 0:00:32.421069
(30, 32, 32, 3)
0.7579053
[Epoch 0/5] [Batch 151/360] [D loss: 0.008154] [G loss: 21.196672] time: 0:00:32.525473
(30, 32, 32, 3)
0.7095763
[Epoch 0/5] [Batch 152/360] [D loss: 0.014576] [G loss: 21.297903] time: 0:00:32.647318
(30, 32, 32, 3)
0.763552
[Epoch 0/5] [Batch 153/360] [D loss: 0.019348] [G loss: 21.537327] time: 0:00:32.756373
(30, 32, 32, 3)
0.8203067
[Epoch 0/5] [Batch 154/360] [D loss: 0.018223] [G loss: 21.428310] time: 0:00:32.866871
(

[Epoch 0/5] [Batch 218/360] [D loss: 0.005054] [G loss: 19.138628] time: 0:00:39.793351
(30, 32, 32, 3)
0.73541576
[Epoch 0/5] [Batch 219/360] [D loss: 0.005148] [G loss: 18.357672] time: 0:00:39.895590
(30, 32, 32, 3)
0.7983858
[Epoch 0/5] [Batch 220/360] [D loss: 0.003981] [G loss: 18.443735] time: 0:00:39.999079
(30, 32, 32, 3)
0.8193615
[Epoch 0/5] [Batch 221/360] [D loss: 0.003389] [G loss: 18.867500] time: 0:00:40.105741
(30, 32, 32, 3)
0.7596182
[Epoch 0/5] [Batch 222/360] [D loss: 0.004641] [G loss: 19.098959] time: 0:00:40.209316
(30, 32, 32, 3)
0.7314741
[Epoch 0/5] [Batch 223/360] [D loss: 0.004533] [G loss: 17.694567] time: 0:00:40.313534
(30, 32, 32, 3)
0.8008949
[Epoch 0/5] [Batch 224/360] [D loss: 0.002381] [G loss: 18.934179] time: 0:00:40.417529
(30, 32, 32, 3)
0.77569574
[Epoch 0/5] [Batch 225/360] [D loss: 0.003825] [G loss: 18.926849] time: 0:00:40.525688
(30, 32, 32, 3)
0.6735309
[Epoch 0/5] [Batch 226/360] [D loss: 0.003943] [G loss: 17.388067] time: 0:00:40.63246

[Epoch 0/5] [Batch 290/360] [D loss: 0.003741] [G loss: 17.422935] time: 0:00:47.527655
(30, 32, 32, 3)
0.7646219
[Epoch 0/5] [Batch 291/360] [D loss: 0.003920] [G loss: 16.434914] time: 0:00:47.631799
(30, 32, 32, 3)
0.78645474
[Epoch 0/5] [Batch 292/360] [D loss: 0.004485] [G loss: 16.521666] time: 0:00:47.735569
(30, 32, 32, 3)
0.80130816
[Epoch 0/5] [Batch 293/360] [D loss: 0.005245] [G loss: 16.924332] time: 0:00:47.844054
(30, 32, 32, 3)
0.66360575
[Epoch 0/5] [Batch 294/360] [D loss: 0.004134] [G loss: 16.376417] time: 0:00:47.949435
(30, 32, 32, 3)
0.7714712
[Epoch 0/5] [Batch 295/360] [D loss: 0.004032] [G loss: 15.805937] time: 0:00:48.055187
(30, 32, 32, 3)
0.78967077
[Epoch 0/5] [Batch 296/360] [D loss: 0.003741] [G loss: 16.168100] time: 0:00:48.159210
(30, 32, 32, 3)
0.7457784
[Epoch 0/5] [Batch 297/360] [D loss: 0.004400] [G loss: 16.249660] time: 0:00:48.263094
(30, 32, 32, 3)
0.74679035
[Epoch 0/5] [Batch 298/360] [D loss: 0.003853] [G loss: 16.285028] time: 0:00:48.37

...

(30, 32, 32, 3)
0.92926425
[Epoch 4/5] [Batch 30/360] [D loss: 0.255281] [G loss: 2.974552] time: 0:02:55.076305
(30, 32, 32, 3)
0.88621205
[Epoch 4/5] [Batch 31/360] [D loss: 0.216594] [G loss: 2.841325] time: 0:02:55.180967
(30, 32, 32, 3)
0.9148225
[Epoch 4/5] [Batch 32/360] [D loss: 0.318035] [G loss: 3.178226] time: 0:02:55.291178
(30, 32, 32, 3)
0.93970627
[Epoch 4/5] [Batch 33/360] [D loss: 0.173133] [G loss: 3.061610] time: 0:02:55.400346
(30, 32, 32, 3)
0.91145796
[Epoch 4/5] [Batch 34/360] [D loss: 0.288793] [G loss: 2.808115] time: 0:02:55.508394
(30, 32, 32, 3)
0.94191617
[Epoch 4/5] [Batch 35/360] [D loss: 0.185525] [G loss: 3.320125] time: 0:02:55.612345
(30, 32, 32, 3)
0.9235103
[Epoch 4/5] [Batch 36/360] [D loss: 0.272858] [G loss: 3.048848] time: 0:02:55.719385
(30, 32, 32, 3)
0.8834438
[Epoch 4/5] [Batch 37/360] [D loss: 0.208488] [G loss: 3.314281] time: 0:02:55.837881
(30, 32, 32, 3)
0.9147837
[Epoch 4/5] [Batch 38/360] [D loss: 0.280720] [G loss: 3.010859] time: 0:

(30, 32, 32, 3)
0.9489948
[Epoch 4/5] [Batch 104/360] [D loss: 0.266585] [G loss: 3.137023] time: 0:03:03.170226
(30, 32, 32, 3)
0.8973231
[Epoch 4/5] [Batch 105/360] [D loss: 0.225490] [G loss: 2.592709] time: 0:03:03.277158
(30, 32, 32, 3)
0.8557143
[Epoch 4/5] [Batch 106/360] [D loss: 0.282883] [G loss: 2.445645] time: 0:03:03.381503
(30, 32, 32, 3)
0.87870336
[Epoch 4/5] [Batch 107/360] [D loss: 0.186522] [G loss: 2.555438] time: 0:03:03.486721
(30, 32, 32, 3)
0.9176037
[Epoch 4/5] [Batch 108/360] [D loss: 0.307527] [G loss: 2.767426] time: 0:03:03.598194
(30, 32, 32, 3)
0.9032931
[Epoch 4/5] [Batch 109/360] [D loss: 0.185502] [G loss: 3.097021] time: 0:03:03.709846
(30, 32, 32, 3)
0.94184417
[Epoch 4/5] [Batch 110/360] [D loss: 0.265737] [G loss: 2.703815] time: 0:03:03.820809
(30, 32, 32, 3)
0.90995806
[Epoch 4/5] [Batch 111/360] [D loss: 0.226576] [G loss: 2.992418] time: 0:03:03.924199
(30, 32, 32, 3)
0.91611046
[Epoch 4/5] [Batch 112/360] [D loss: 0.241522] [G loss: 3.107934] 

(30, 32, 32, 3)
0.916311
[Epoch 4/5] [Batch 178/360] [D loss: 0.240217] [G loss: 2.483938] time: 0:03:11.139273
(30, 32, 32, 3)
0.9758255
[Epoch 4/5] [Batch 179/360] [D loss: 0.268129] [G loss: 2.675939] time: 0:03:11.241610
(30, 32, 32, 3)
0.93183464
[Epoch 4/5] [Batch 180/360] [D loss: 0.234047] [G loss: 2.542035] time: 0:03:11.347156
(30, 32, 32, 3)
0.9060548
[Epoch 4/5] [Batch 181/360] [D loss: 0.257742] [G loss: 2.567840] time: 0:03:11.454518
(30, 32, 32, 3)
0.9470427
[Epoch 4/5] [Batch 182/360] [D loss: 0.233213] [G loss: 2.393216] time: 0:03:11.563775
(30, 32, 32, 3)
0.8934727
[Epoch 4/5] [Batch 183/360] [D loss: 0.248610] [G loss: 2.626778] time: 0:03:11.668492
(30, 32, 32, 3)
0.87358856
[Epoch 4/5] [Batch 184/360] [D loss: 0.223634] [G loss: 2.400700] time: 0:03:11.783379
(30, 32, 32, 3)
0.89474505
[Epoch 4/5] [Batch 185/360] [D loss: 0.232941] [G loss: 2.665513] time: 0:03:11.888249
(30, 32, 32, 3)
0.9051855
[Epoch 4/5] [Batch 186/360] [D loss: 0.231050] [G loss: 2.491016] ti

(30, 32, 32, 3)
0.94494915
[Epoch 4/5] [Batch 252/360] [D loss: 0.252120] [G loss: 2.424499] time: 0:03:19.091470
(30, 32, 32, 3)
0.9268792
[Epoch 4/5] [Batch 253/360] [D loss: 0.266996] [G loss: 2.318520] time: 0:03:19.199163
(30, 32, 32, 3)
0.9420886
[Epoch 4/5] [Batch 254/360] [D loss: 0.229063] [G loss: 2.435534] time: 0:03:19.308003
(30, 32, 32, 3)
0.916913
[Epoch 4/5] [Batch 255/360] [D loss: 0.249932] [G loss: 2.314553] time: 0:03:19.412295
(30, 32, 32, 3)
0.94497633
[Epoch 4/5] [Batch 256/360] [D loss: 0.228311] [G loss: 2.172278] time: 0:03:19.518632
(30, 32, 32, 3)
0.90199727
[Epoch 4/5] [Batch 257/360] [D loss: 0.239331] [G loss: 2.017995] time: 0:03:19.626408
(30, 32, 32, 3)
0.8640971
[Epoch 4/5] [Batch 258/360] [D loss: 0.231280] [G loss: 2.140234] time: 0:03:19.736607
(30, 32, 32, 3)
0.9337077
[Epoch 4/5] [Batch 259/360] [D loss: 0.212888] [G loss: 2.356901] time: 0:03:19.841343
(30, 32, 32, 3)
0.9068525
[Epoch 4/5] [Batch 260/360] [D loss: 0.260373] [G loss: 2.401643] ti

(30, 32, 32, 3)
0.9334894
[Epoch 4/5] [Batch 326/360] [D loss: 0.253511] [G loss: 2.586312] time: 0:03:27.131690
(30, 32, 32, 3)
0.8869097
[Epoch 4/5] [Batch 327/360] [D loss: 0.263157] [G loss: 2.397327] time: 0:03:27.240774
(30, 32, 32, 3)
0.9348299
[Epoch 4/5] [Batch 328/360] [D loss: 0.254202] [G loss: 2.276065] time: 0:03:27.345229
(30, 32, 32, 3)
0.905732
[Epoch 4/5] [Batch 329/360] [D loss: 0.254366] [G loss: 2.216895] time: 0:03:27.448055
(30, 32, 32, 3)
0.9259682
[Epoch 4/5] [Batch 330/360] [D loss: 0.271789] [G loss: 2.299426] time: 0:03:27.556910
(30, 32, 32, 3)
0.8993947
[Epoch 4/5] [Batch 331/360] [D loss: 0.236714] [G loss: 2.502393] time: 0:03:27.664138
(30, 32, 32, 3)
0.9155021
[Epoch 4/5] [Batch 332/360] [D loss: 0.277089] [G loss: 2.262060] time: 0:03:27.770919
(30, 32, 32, 3)
0.88067466
[Epoch 4/5] [Batch 333/360] [D loss: 0.232050] [G loss: 2.257247] time: 0:03:27.874623
(30, 32, 32, 3)
0.92977285
[Epoch 4/5] [Batch 334/360] [D loss: 0.252297] [G loss: 2.213200] tim

9) We can test the model with 100 test data which will be saved as images

In [11]:
for batch_i in range(100):
    test_first_imgs, test_last_imgs,_ = next(test_batch_generator)
    test_fake_last_imgs = modelObj.generator.predict(test_first_imgs) 

    test_img_name = output_log_dir + "/gen_img_test_" + str(batch_i) + ".png"
    merged_img = np.vstack((test_first_imgs[0],test_last_imgs[0],test_fake_last_imgs[0]))
    imageio.imwrite(test_img_name, img_as_ubyte(merged_img))

## EXERCISES

#### Exercise 1)
Update the network architecture given in  **build_generator**  and  **build_discriminator**  of the class GANModel. Please note that the current image resolution is set to 32x32 (i.e. IMAGE_WIDTH and IMAGE_HEIGHT values) in the file configGAN.py. 
This way initial experiements can run faster. Once you implement the inital version of the network, please set the resolution values back to 128x128. Experimental results should be provided for this high resolution images.  

**Hint:** As a generator model, you can use the segmentation model implemented in lab03. Do not forget to adapt the input and output shapes of the generator model in this case.

#### Exercise 2) 
Use different **optimization** (e.g. ADAM, SGD, etc) and **regularization** (e.g. data augmentation, dropout) methods to increase the network accuracy. 